# Kings County Housing Price Prediction

## Overview

Kings County, often referred to as Brooklyn, is one of the most popular neighborhood in United States. Therefore, evaluating and predicting housing prices accurately provides. immense value for all participants of real estate transaction in this neighborhood. This project seeks to analyze and model the given Kings County real estate data to predict future sale prices of different properties. 